# 📊 Final Data Visualisation

This module creates polished charts from the cleaned Airbnb dataset and saves them into a dedicated `charts/` folder. It follows the cleaning stage and uses only the final, standardised data.

## What this module does
- Loads the cleaned dataset for visual analysis.
- Caps extreme values (95th percentile) so charts are realistic and readable.
- Standardises neighbourhood names to remove inconsistencies.
- Generates three final charts:
  1. **Average price by room type**
  2. **Average price by neighbourhood group**
  3. **Top 10% price distribution (violin plot)**

Each chart is displayed on screen and exported as a `.png` file for portfolio use.

## Why this matters for a portfolio
These visuals demonstrate:
- Clean handling of real-world messiness (spelling issues, extreme values).
- Ability to produce clear, professional data graphics.
- A reproducible process: load → clean → visualise.

## How to run
```bash
python visualisation.py


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:

# Relative path to cleaned data
CLEAN_PATH = os.path.join("data", "Airbnb_Open_Data.csv")

def load_clean_data(path):
    """Load the cleaned Airbnb dataset."""
    df = pd.read_csv(path)
    print(f"Loaded cleaned data: {path} | Rows: {len(df)}, Columns: {len(df.columns)}")
    return df



In [ ]:
def create_final_charts(clean_df):
    """Create polished charts and save them to the charts/ folder."""

    # Ensure charts folder exists
    os.makedirs('charts', exist_ok=True)

    # ----------------------------
    # Standardise text categories
    # ----------------------------
    if 'neighbourhood_group' in clean_df.columns:
        clean_df['neighbourhood_group'] = (
            clean_df['neighbourhood_group']
            .str.lower()
            .str.strip()
        )
        clean_df['neighbourhood_group'].replace({
            'manhatan': 'manhattan',
            'brookln': 'brooklyn',
        }, inplace=True)

    # ----------------------------
    # Cap extreme values for visuals
    # ----------------------------
    cap_price = clean_df['price'].quantile(0.95)
    capped_df = clean_df[clean_df['price'] <= cap_price]

    # -----------------------------------------
    # 1️⃣ Average price by room type
    # -----------------------------------------
    if {'room_type', 'price'}.issubset(clean_df.columns):
        avg_price_by_room = (
            capped_df.groupby('room_type')['price']
            .mean()
            .reset_index()
        )

        plt.figure(figsize=(6, 4))
        sns.barplot(
            x='room_type',
            y='price',
            data=avg_price_by_room,
            palette='coolwarm'
        )
        plt.title('Average Price by Room Type (95th percentile)')
        plt.ylabel('Average Price ($)')
        plt.xlabel('Room Type')
        plt.ylim(
            avg_price_by_room['price'].min() * 0.95,
            avg_price_by_room['price'].max() * 1.05
        )
        plt.tight_layout()
        plt.savefig('charts/avg_price_room_type.png')
        plt.show()

    # -----------------------------------------
    # 2️⃣ Average price by neighbourhood group
    # -----------------------------------------
    if {'neighbourhood_group', 'price'}.issubset(clean_df.columns):
        neigh_df = capped_df[
            capped_df['neighbourhood_group'] != 'missing'
        ]

        avg_price_by_neigh = (
            neigh_df.groupby('neighbourhood_group')['price']
            .mean()
            .reset_index()
        )

        plt.figure(figsize=(6, 4))
        sns.barplot(
            x='neighbourhood_group',
            y='price',
            data=avg_price_by_neigh,
            palette='viridis'
        )
        plt.title('Average Price by Neighbourhood Group (95th percentile)')
        plt.ylabel('Average Price ($)')
        plt.xlabel('Neighbourhood')
        plt.ylim(
            avg_price_by_neigh['price'].min() * 0.95,
            avg_price_by_neigh['price'].max() * 1.05
        )
        plt.tight_layout()
        plt.savefig('charts/avg_price_neighbourhood.png')
        plt.show()

    # -----------------------------------------
    # 3️⃣ Top 10% listings — violin plot
    # -----------------------------------------
    if 'price' in clean_df.columns:
        top_10pct = clean_df[
            clean_df['price'] >= clean_df['price'].quantile(0.90)
        ]
        top_10pct = top_10pct.dropna(
            subset=['neighbourhood_group', 'room_type']
        )

        plt.figure(figsize=(8, 5))
        sns.violinplot(
            x='neighbourhood_group',
            y='price',
            data=top_10pct,
            palette='Set2',
            inner='box',
            cut=0
        )

        plt.title('Price Distribution of Top 10% Listings')
        plt.ylabel('Price (£)')
        plt.xlabel('Neighbourhood')

        plt.ylim(
            1150,
            top_10pct['price'].max() * 1.05
        )
        plt.tight_layout()
        plt.savefig('charts/top10pct_violin.png')
        plt.show()

    print("All final charts created and saved to charts/ folder.")

In [ ]:
if __name__ == "__main__":
    df = load_clean_data()
    create_final_charts(df)
